In [6]:
%pip install scikit-learn --quiet

You should consider upgrading via the '/Users/xavierchavez/.pyenv/versions/3.9.5/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd


In [2]:
import re



In [18]:


# Función para preprocesar los textos y dividirlos en términos
def preprocess_text(text):
    if text == 'No hay informacion':
        return ''
    # Divide el texto en términos usando saltos de línea como separadores
    terms = re.split(r'\n+', text)
    return ' '.join(terms)  # Une los términos con espacios


# Cargar el archivo CSV
df = pd.read_csv('data_pdf_join.csv')

# Seleccionar las columnas relevantes (nombreCarrera, nombreMateria, CONT_MINIMOS)
df = df[['nombreCarrera', 'nombreMateria', 'CONT_MINIMOS', 'CURSO_MALLA_CODIGO', 'CURSO_MALLA_CICLO', 'MODALIDAD_CODIGO', 'MODALIDAD_NOMBRE', 'RESPONSABLE_PROGRAMA_NOMBRE']]

# Aplicar el preprocesamiento a los contenidos mínimos
df['CONT_MINIMOS'] = df['CONT_MINIMOS'].apply(preprocess_text)

# Inicializar el vectorizador TF-IDF sin filtrar stop words
tfidf_vectorizer = TfidfVectorizer(stop_words=None)

# Crear un DataFrame para almacenar las similitudes
similarity_df = pd.DataFrame(columns=['Carrera1', 'Materia1', 'Carrera2', 'Facultad1', 'Facultad2',
                             'Materia2', 'Similitud', 'Contenidos1', 'Codigo1', 'Ciclo1', 'Modalidad1','Contenidos2', 'Codigo2', 'Ciclo2', 'Modalidad2'])

# Iterar sobre cada carrera
for carrera1 in df['nombreCarrera'].unique():
    # Filtrar las materias de la carrera actual
    materias_carrera1 = df[(df['nombreCarrera'] == carrera1)
                           & (df['CONT_MINIMOS'] != '')]

    if not materias_carrera1.empty:
        # Obtener la matriz TF-IDF de los contenidos mínimos de las materias
        tfidf_matrix1 = tfidf_vectorizer.fit_transform(
            materias_carrera1['CONT_MINIMOS'])

        # Calcular la similitud de coseno entre todas las combinaciones de materias
        similarity_matrix1 = cosine_similarity(tfidf_matrix1)

        # Obtener los nombres de las materias
        materias1 = materias_carrera1['nombreMateria'].values
        contenidos_carrera1 = materias_carrera1['CONT_MINIMOS'].values
        codigos_carrera1 = materias_carrera1['CURSO_MALLA_CODIGO'].values  # Agregado
        curso_ciclo = materias_carrera1['CURSO_MALLA_CICLO'].values  # Agregado
        modalidad_carrera1 = materias_carrera1['MODALIDAD_NOMBRE'].values  # Agregado
        facultad_carrera1 = materias_carrera1['RESPONSABLE_PROGRAMA_NOMBRE'].values  # Agregado
        # Crear un DataFrame temporal para almacenar las similitudes
        temp_df1 = pd.DataFrame(
            similarity_matrix1, columns=materias1, index=materias1)

        # Iterar sobre cada otra carrera para comparar
        for carrera2 in df['nombreCarrera'].unique():
            # Filtrar las materias de la otra carrera
            materias_carrera2 = df[(df['nombreCarrera'] == carrera2) & (
                df['CONT_MINIMOS'] != '')]

            if not materias_carrera2.empty and carrera1 != carrera2:
                # Obtener la matriz TF-IDF de los contenidos mínimos de las materias de la otra carrera
                tfidf_matrix2 = tfidf_vectorizer.transform(
                    materias_carrera2['CONT_MINIMOS'])

                # Calcular la similitud de coseno entre todas las combinaciones de materias
                similarity_matrix2 = cosine_similarity(
                    tfidf_matrix1, tfidf_matrix2)

                # Obtener los nombres de las materias de la otra carrera
                materias2 = materias_carrera2['nombreMateria'].values
                ciclo2 = materias_carrera2['CURSO_MALLA_CICLO'].values  # Agregado

                contenidos_carrera2 = materias_carrera2['CONT_MINIMOS'].values
                codigos_carrera2 = materias_carrera2['CURSO_MALLA_CODIGO'].values  # Agregado
                modalidad_carrera2 = materias_carrera2['MODALIDAD_NOMBRE'].values  # Agregado
                facultad_carrera2 = materias_carrera2['RESPONSABLE_PROGRAMA_NOMBRE'].values  # Agregado

                # Crear un DataFrame temporal para almacenar las similitudes
                temp_df2 = pd.DataFrame(
                    similarity_matrix2, columns=materias2, index=materias1)

                # Convertir la matriz de similitud en un DataFrame
                temp_df2 = temp_df2.stack().reset_index()
                temp_df2.columns = ['Materia1', 'Materia2', 'Similitud']
                temp_df2['Carrera1'] = carrera1
                temp_df2['Carrera2'] = carrera2
    
                # Agregar los contenidos mínimos de la primera carrera
                temp_df2['Contenidos1'] = temp_df2['Materia1'].apply(
                    lambda x: contenidos_carrera1[materias1 == x][0])
                
                temp_df2['Codigo1'] = temp_df2['Materia1'].apply(
                    lambda x: codigos_carrera1[materias1 == x][0])
                
                temp_df2['Ciclo1'] = temp_df2['Materia1'].apply(
                    lambda x: curso_ciclo[materias1 == x][0])
                
                temp_df2['Modalidad1'] = temp_df2['Materia1'].apply(
                    lambda x: modalidad_carrera1[materias1 == x][0]
                )
                temp_df2['Facultad1'] = temp_df2['Materia1'].apply(
                    lambda x: facultad_carrera1[materias1 == x][0]
                )

            
                # Agregar los contenidos mínimos de la segunda carrera
                temp_df2['Contenidos2'] = temp_df2['Materia2'].apply(
                    lambda x: contenidos_carrera2[materias2 == x][0])
                
                temp_df2['Codigo2'] = temp_df2['Materia2'].apply(
                    lambda x: codigos_carrera2[materias2 == x][0])
                
                temp_df2['Ciclo2'] = temp_df2['Materia2'].apply(
                    lambda x: ciclo2[materias2 == x][0])
            
                temp_df2['Modalidad2'] = temp_df2['Materia2'].apply(
                    lambda x: modalidad_carrera2[materias2 == x][0]
                )
                temp_df2['Facultad2'] = temp_df2['Materia2'].apply(
                    lambda x: facultad_carrera2[materias2 == x][0]
                )
                
                # Convertir los valores de similitud a porcentaje
                temp_df2['Similitud'] = (
                    temp_df2['Similitud'] * 100).round(2).astype(str) + '%'
                    

                # Agregar las similitudes al DataFrame principal
                similarity_df = pd.concat(
                    [similarity_df, temp_df2], ignore_index=True)

# Reordenar las columnas
similarity_df = similarity_df[['Carrera1', 'Materia1', 'Codigo1', 'Ciclo1', 'Contenidos1', 'Modalidad1', 'Facultad1',
                               'Carrera2', 'Materia2', 'Codigo2', 'Ciclo2', 'Contenidos2', 'Modalidad2', 'Facultad2', 'Similitud']]

# Exportar el DataFrame a un archivo CSV
similarity_df.to_csv('similitud_materias_entre_carreras.csv', index=False)
